In [2]:
import pandas as pd
from tensorflow import keras
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, precision_score, recall_score
from scikeras.wrappers import KerasClassifier
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [3]:
df = pd.read_csv("dades/points_data.csv")

In [4]:
# Select quantitatives variables
quant_var = df[["altitut", "pendent", "orientacio", "t_max", "u", "v", "specific_humidity", "relative_humidity"]]
# Normalize
mean = quant_var.mean()
mean.to_csv("dades/means.csv")
std = quant_var.std()
std.to_csv("dades/stds.csv")
normalized_df = (quant_var-quant_var.mean())/quant_var.std()
# Add soil use binarizaded
normalized_df = normalized_df.join(pd.get_dummies(df["coberta_sol"].astype("category")))

In [5]:
# Split dataset
y = df["fire"]
xTrain, xTest, yTrain, yTest = train_test_split(normalized_df, y, test_size=0.2, random_state=12)

In [6]:
# Create function returning a model
def design_model (dropout_rate, weight_constraint):

    model = keras.Sequential()

    model.add(keras.layers.Dense(64, input_shape=(19,), activation='relu', kernel_constraint=keras.constraints.MaxNorm(weight_constraint)))
    model.add(keras.layers.Dropout(dropout_rate))

    model.add(keras.layers.Dense(128, activation='relu', kernel_constraint=keras.constraints.MaxNorm(weight_constraint)))
    model.add(keras.layers.Dropout(dropout_rate))

    model.add(keras.layers.Dense(256, activation='relu', kernel_constraint=keras.constraints.MaxNorm(weight_constraint)))
    model.add(keras.layers.Dropout(dropout_rate))

    model.add(keras.layers.Dense(128, activation='relu', kernel_constraint=keras.constraints.MaxNorm(weight_constraint)))
    model.add(keras.layers.Dropout(dropout_rate))

    model.add(keras.layers.Dense(64, activation='relu', kernel_constraint=keras.constraints.MaxNorm(weight_constraint)))
    model.add(keras.layers.Dropout(dropout_rate))
    
    model.add(keras.layers.Dense(2, activation='softmax'))
    
    model.compile(loss="sparse_categorical_crossentropy", metrics=["accuracy"])

    return model

model = KerasClassifier(model=design_model, verbose=0)

In [7]:
# Define the search parameters
epochs = [10,20,50,100]
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
weight_constraint = [1.0, 2.0, 3.0, 4.0, 5.0]
dropout_rate = [ 0.2, 0.3, 0.4, 0.5]

In [8]:
param_grid = dict(model__dropout_rate=dropout_rate, model__weight_constraint=weight_constraint, optimizer=optimizer, epochs=epochs)

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(xTrain, yTrain)

In [9]:
# Print results
print(f'Best:{grid_result.best_score_} using {grid_result.best_params_}')

for mean, stdev, param in zip(grid_result.cv_results_['mean_test_score'], 
                              grid_result.cv_results_['std_test_score'], grid_result.cv_results_['params']):
    print(f"{mean} {stdev} with: {param}")

Best:0.9276654238470708 using {'epochs': 50, 'model__dropout_rate': 0.3, 'model__weight_constraint': 1.0, 'optimizer': 'Adamax'}
0.9240174245330143 0.006086959783168933 with: {'epochs': 10, 'model__dropout_rate': 0.2, 'model__weight_constraint': 1.0, 'optimizer': 'SGD'}
0.9238587699240979 0.008750663752861787 with: {'epochs': 10, 'model__dropout_rate': 0.2, 'model__weight_constraint': 1.0, 'optimizer': 'RMSprop'}
0.9252859821029751 0.0057973453511524315 with: {'epochs': 10, 'model__dropout_rate': 0.2, 'model__weight_constraint': 1.0, 'optimizer': 'Adagrad'}
0.9233828815752787 0.005082120367766333 with: {'epochs': 10, 'model__dropout_rate': 0.2, 'model__weight_constraint': 1.0, 'optimizer': 'Adadelta'}
0.9235414607062652 0.005432499612678285 with: {'epochs': 10, 'model__dropout_rate': 0.2, 'model__weight_constraint': 1.0, 'optimizer': 'Adam'}
0.9229063894030194 0.006738838658532172 with: {'epochs': 10, 'model__dropout_rate': 0.2, 'model__weight_constraint': 1.0, 'optimizer': 'Adamax'}
0

In [13]:
##### Design final model with best parameters
model = keras.Sequential()

model.add(keras.layers.Dense(64, input_shape=(19,), activation='relu', kernel_constraint=keras.constraints.MaxNorm(1)))
model.add(keras.layers.Dropout(0.3))

model.add(keras.layers.Dense(128, activation='relu', kernel_constraint=keras.constraints.MaxNorm(1)))
model.add(keras.layers.Dropout(0.3))

model.add(keras.layers.Dense(256, activation='relu', kernel_constraint=keras.constraints.MaxNorm(1)))
model.add(keras.layers.Dropout(0.3))

model.add(keras.layers.Dense(128, activation='relu', kernel_constraint=keras.constraints.MaxNorm(1)))
model.add(keras.layers.Dropout(0.3))

model.add(keras.layers.Dense(64, activation='relu', kernel_constraint=keras.constraints.MaxNorm(1)))
model.add(keras.layers.Dropout(0.3))
    
model.add(keras.layers.Dense(2, activation='softmax'))

In [14]:
# Define the accuracy metrics and parameters
model.compile(optimizer="Adamax", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# Run the model
model.fit(xTrain, yTrain, epochs=50)

# Predict for test data 
yTestPredicted = model.predict(xTest)
yTestPredicted = yTestPredicted[:,1]

# Calculate the error metrics
yTestPredicted = (yTestPredicted>0.5).astype(int)
cMatrix = confusion_matrix(yTest, yTestPredicted)
pScore = precision_score(yTest, yTestPredicted)
rScore = recall_score(yTest, yTestPredicted)

print("\nConfusion matrix:", cMatrix)
print("\nP-Score: %.3f, R-Score: %.3f" % (pScore, rScore))

Epoch 1/50
197/197 [==============================] - 1s 3ms/step - loss: 0.4001 - accuracy: 0.8258
Epoch 2/50
197/197 [==============================] - 1s 3ms/step - loss: 0.3017 - accuracy: 0.8877
Epoch 3/50
197/197 [==============================] - 1s 3ms/step - loss: 0.2839 - accuracy: 0.8904
Epoch 4/50
197/197 [==============================] - 0s 2ms/step - loss: 0.2706 - accuracy: 0.8997
Epoch 5/50
197/197 [==============================] - 0s 2ms/step - loss: 0.2593 - accuracy: 0.9024
Epoch 6/50
197/197 [==============================] - 0s 2ms/step - loss: 0.2526 - accuracy: 0.9085
Epoch 7/50
197/197 [==============================] - 0s 2ms/step - loss: 0.2456 - accuracy: 0.9097
Epoch 8/50
197/197 [==============================] - 0s 2ms/step - loss: 0.2392 - accuracy: 0.9135
Epoch 9/50
197/197 [==============================] - 0s 2ms/step - loss: 0.2416 - accuracy: 0.9134
Epoch 10/50
197/197 [==============================] - 1s 3ms/step - loss: 0.2350 - accuracy: 0.9124

In [15]:
# Export model
model.save("dades/nn_fire_risk.h5")